In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

# Load the CSV file to inspect its contents
file_path = '/content/drive/MyDrive/test/ICCKE-2024/keyword_suggestions_end.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the file to understand its structure
data.head()


,Keyword,Suggestion
0,Per- and polyfluoroalkyl substances,Per- and polyfluoroalkyl |substances 2016
1,Per- and polyfluoroalkyl substances,Per- and polyfluoroalkyl |substances classific...
2,Per- and polyfluoroalkyl substances,Per- and polyfluoroalkyl |substances biodegrad...
3,Per- and polyfluoroalkyl substances,Per- and polyfluoroalkyl |substances hulis
4,Per- and polyfluoroalkyl substances,Per- and polyfluoroalkyl |substances youth


In [4]:
# Extract all words from both columns
keywords = data['Keyword'].str.split(expand=True).stack().unique()
suggestions = data['Suggestion'].str.split(expand=True).stack().unique()

# Combine and extract unique words from both lists
all_words = pd.Series(list(set(keywords) | set(suggestions)))

all_words.head(20)  # Displaying the first 20 unique words for review

,0
0,removal
1,women
2,rules
3,strategy
4,integration
5,harm
6,dcs
7,introduction
8,|substances
9,chemistry


In [5]:
!pip install ace_tools


In [6]:
#pip -q install torch


In [7]:
!pip -q install ace
import ace

from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer
import torch

# Load a pre-trained BERT model and tokenizer specialized in scientific text
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define a function to get the embedding for a given text
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [8]:
# Example chemical ontology terms for comparison
chemical_terms = ['polyfluoroalkyl',"chemical", "compound", "molecule", "reaction", "element", "organic", "inorganic", "acid", "base", "solvent", "catalyst", "polymer", "alkane", "alkene", "alkyne", "aromatic", "amine", "carbon", "oxygen", "hydrogen"]

# Get embeddings for chemical terms
chemical_embeddings = {term: get_embedding(term) for term in chemical_terms}

# Calculate similarity scores for each word in the extracted list against the chemical ontology
similarity_scores = []

for word in all_words:
    word_embedding = get_embedding(word)
    similarities = {term: cosine_similarity(word_embedding.unsqueeze(0), chem_emb.unsqueeze(0)).item() for term, chem_emb in chemical_embeddings.items()}
    max_term = max(similarities, key=similarities.get)
    similarity_scores.append({
        'Word': word,
        'Closest_Chemical_Term': max_term,
        'Similarity_Score': similarities[max_term]
    })

# Convert the results to a DataFrame and display them
similarity_df = pd.DataFrame(similarity_scores).sort_values(by="Similarity_Score", ascending=False)

similarity_df.head(20)  # Displaying the top 20 results

,Word,Closest_Chemical_Term,Similarity_Score
107,reaction,reaction,1.000000
20,polyfluoroalkyl,polyfluoroalkyl,1.000000
124,hazardous,organic,0.914544
104,soil,organic,0.887644
81,nicotine,organic,0.885931
129,food,organic,0.885442
40,wine,organic,0.883475
126,biofilm,polymer,0.880755
73,formation,hydrogen,0.880230
0,removal,carbon,0.868528


In [13]:
# Convert the results to a DataFrame and display them
similarity_df = pd.DataFrame(similarity_scores).sort_values(by="Similarity_Score", ascending=False)

similarity_df.head(20)  # Displaying the top 20 results

,Word,Closest_Chemical_Term,Similarity_Score
107,reaction,reaction,1.000000
20,polyfluoroalkyl,polyfluoroalkyl,1.000000
124,hazardous,organic,0.914544
104,soil,organic,0.887644
81,nicotine,organic,0.885931
129,food,organic,0.885442
40,wine,organic,0.883475
126,biofilm,polymer,0.880755
73,formation,hydrogen,0.880230
0,removal,carbon,0.868528


In [15]:
similarity_df [similarity_df['Similarity_Score'] > 0.9]
similarity_df.to_csv('/content/drive/MyDrive/test/ICCKE-2024/similarity_results.csv', index=False)